In [3]:
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
import tensorflow as tf
import cv2


In [8]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_variable("kernel",
                                    shape=[int(input_shape[-1]),
                                           self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

In [9]:
layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

Instructions for updating:
Please use `layer.add_weight` method instead.
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.27940702, -0.0935185 , -0.40345687,  0.0896666 , -0.09473616,
        -0.56032944, -0.4388228 , -0.17537156, -0.53281903, -0.11914891],
       [ 0.15512401,  0.31326157,  0.3228491 ,  0.5446457 , -0.04062986,
        -0.10492331, -0.10741496,  0.622472  , -0.3152505 ,  0.11353219],
       [ 0.03874904, -0.6035432 ,  0.561742  , -0.3990659 , -0.03816158,
        -0.50459146, -0.21395653,  0.41864496, -0.4892007 ,  0.03045398],
       [ 0.29590046, -0.24804816,  0

In [15]:
class FreeSpacePropagation(tf.keras.layers.Layer):
    def __init__(self, output_dim, pitch_size, z, k):
        super(FreeSpacePropagation, self).__init__()
        self.output_dim = output_dim
        self.pitch_size = pitch_size
        self.z = z
        self.k = k

    def build(self, input_shape):
        x1 = np.arange(0, input_shape[1], 1)
        y1 = np.arange(0, input_shape[0], 1)
        xx1, yy1 = np.meshgrid(x1, y1)
        xx1 = xx1.reshape(1, -1)
        yy1 = yy1.reshape(1, -1)

        x2 = np.arange(0, self.output_dim[1] * 2, 0.5)
        y2 = np.arange(0, self.output_dim[0] * 2, 0.5)
        xx2, yy2 = np.meshgrid(x2, y2)
        xx2 = xx2.reshape(-1, 1)
        yy2 = yy2.reshape(-1, 1)

        dx = self.pitch_size*(xx1 - xx2)
        dy = self.pitch_size*(yy1 - yy2)
        r = np.sqrt(dx**2 + dy**2 + self.z**2)
        w = 1/(2*np.pi) * self.z / r * (1/r - 1j*self.k) * np.exp(1j * self.k * r)
        self.propagation_matrix = tf.Variable(initial_value=w,
                                              trainable=False)

    def call(self, inputs, **kwargs):
        _input = tf.reshape(inputs, (-1,1))
        _out = tf.matmul(_input, self.propagation_matrix    )
        return tf.reshape(_out, self.output_dim)

In [16]:
l = 633e-9
k = 2 * np.pi / l
d = 1e-6
z = 1e-5
shape = (28, 28)
propagation = FreeSpacePropagation(shape, d, z, k)
pattern = np.zeros(shape)
pattern[0::3, 0::3] = 1
out = propagation(pattern)
plt.imshow(pattern)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



InvalidArgumentError: cannot compute _MklMatMul as input #1(zero-based) was expected to be a float tensor but is a complex128 tensor [Op:MatMul] name: free_space_propagation_5/MatMul/